In [1]:
import tensorflow as tf
import numpy as np 
import gym 
import networkx as nx
import math 
#from tensorflow import keras


In [2]:
#proviamo prima una fully connected
n = 3
m = 4
k=n*m
RailCost=0
StopCost=0

#la mappa è un lattice i cui nodi rappresentano le POSSIBILI stazioni 
MapMatrix= np.zeros(shape=( n, m))

#La così detta amtrice verdhttp://localhost:8888/notebooks/Metropolitan.py.ipynb#e, dato che vogliamo usare keras, possiamo generarla a priori un numero arbitrario di volte e trattare il problema come semi-supervisionato
#in questo caso le sezioni di TravelBank sono le TravelMatrix
NumberOfCities=1000
TravelBank=np.zeros(shape=(NumberOfCities,n*m,n*m,))
for s in range(NumberOfCities):
    TravelBank[s] = np.random.rand(n*m, n*m)
    for i in range(k):
        somma=TravelBank[s,i].sum()
        for j in range(k):
            TravelBank[s,i,j]=TravelBank[s,i,j]/somma#costruisce matrici con colonne normalizzate.            
def AssignCoordinates(MapMatrix,v):#v is a number associated to the vth node in the network 
    #assigns coordinates to a node in a map 
    n=len(MapMatrix)
    m=len(MapMatrix[0])
    coord=np.zeros(shape=(2))
    coord[0]=v%n  #x of v 
    coord[1]=math.floor(v/n)  #y of v #in coordinate eculideee a aprtire da 0
    return coord
def EuclideanDist(MapMatrix,v,w):
    W_coords=np.zeros(shape=(2))
    V_coords=np.zeros(shape=(2))
    W_coords=AssignCoordinates(MapMatrix,w)
    V_coords=AssignCoordinates(MapMatrix,v)
    return math.sqrt((W_coords[0]-V_coords[0]+1)*(W_coords[0]-V_coords[0]+1)+(W_coords[1]-V_coords[1]+1)*(W_coords[1]-V_coords[1]+1))


StateMatrix = np.zeros(shape=(n*m, n*m))
for i in range(k):
    for j in range(k):
           if(i != j):
                 StateMatrix[i,j]=50



def RewardFunction(MapMatrix,TravelMatrix,StateMatrix,RailCost,StopCost):
    #StateMatrix1=np.zeros(shape=(k,k))
    Reward=0
    StateMatrix=np.reshape(StateMatrix,(k,k))
    for i in range(k):
        for j in range((k)):
            StateMatrix[i,j]=StateMatrix[i,j]*EuclideanDist(MapMatrix,i,j)#StateMatrix diventa una matrice che contiene la distanza euclidea nelle entrate
    for i in range(k):
        for j in range((k)):
            if(i!=j):
                Reward+=(TravelMatrix[i,j])/(Dijkstra(StateMatrix,i,j)) - RailCost - StopCost
    return Reward 





In [3]:
def Dijkstra(StateMatrix,source,target):
#l'algo calcola un array di distanze(una per ogni nodo del grafo
    Distances=np.array([])
    Queue=np.array([])
    Visited=np.array([])
    for s in range(k):
        if(s != source):
            Distances=np.append(Distances,100000000) 
        else:
            Distances=np.append(Distances,0) 
        Queue=np.append(Queue,s)
    v=0
    switch=1
    while(len(Queue)!=0 and switch):
        #print(Queue)
        v=int (int (np.where(Distances == np.sort(Distances)[len(Visited)])[0]))#devo escludere dalle distanze quell ivisitati?
        if(v not in Visited):
            Queue=np.delete(Queue,int (np.where(Queue==int (np.where(Distances == np.sort(Distances)[len(Visited)])[0]))[0]))
            Visited=np.append(Visited,v)
        for u in range(k):
            if(StateMatrix[v,u]!=0):
                tempDist=Distances[v]+StateMatrix[v,u]
                 
                if(tempDist<Distances[u]):
                    Distances[u]=tempDist
    #print(Distances)
    return Distances[target]

#testing
summing=0
for i in range(NumberOfCities):
    if(i%100==0):
        print(i)
    summing+=RewardFunction(MapMatrix,TravelBank[i],TravelBank[i],0,0)
print(summing/NumberOfCities)

0
100
200
300
400
500
600
700
800
900
340.73670976432265
